In [1]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [2]:
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())

In [3]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

hamiltonian = np.zeros((2**10, 2**10), dtype=np.cdouble)
# comment this out when stuff actually works
coefficients = [1]*1
bases = ['XIIIIIIIII']# 'XXXXIIIIII', 'XXIIIIIIII', 'XXXIIIIIII'] # these are all commuting
# numTerms = len(bases)
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

In [4]:
is_hermitian(hamiltonian)

0.0

In [7]:

def get_circuit(t=1, n=1):
    n = 1 # number of time divisions. n=1 works for commuting operators 
    qc = QuantumCircuit(10)
    for i in range(n):
        for c, b in zip(coefficients, bases):
            # implement e^{-i*c*b*t/n}
            print(c, b)
            q = []
            qc.barrier()
            for i, op in enumerate(b):
                if (op == 'X'):
                    qc.h(i)
                    q.append(i)
                elif (op == 'Z'):
                    q.append(i)
                elif (op == 'Y'):
                    qc.h(i)
                    qc.s(i)
                    q.append(i)
                elif (op == 'I'):
                    continue
            if (len(q) > 0):
                if (len(q) > 1):
                    for i in range(len(q)-1):
                        qc.cx(q[i], q[-1])
                qc.rz(2*c*t/n, q[-1])
                if (len(q) > 1):
                    for i in reversed(range(len(q)-1)):
                        qc.cx(q[i], q[-1])
                for i, op in enumerate(b):
                    if (op == 'X'):
                        qc.h(i)
                    elif (op == 'Z'):
                        continue
                    elif (op == 'Y'):
                        qc.sdg(i)
                        qc.h(i)
                    elif (op == 'I'):
                        continue
            qc.barrier()
        return qc

def get_unitaries(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [9]:
e_ith4, U_circuit4 = get_unitaries()
e_ith, U_circuit = get_unitaries(t=0.25)

1 XIIIIIIIII
1 XIIIIIIIII


In [10]:
def get_error(e_ith, U_circuit):
    diff = U_circuit - e_ith
    # measure the operator's accuracy
    data = diff.data
    w, v = LA.eig(data)

    return np.absolute(max(w))

In [11]:
get_error(e_ith4, U_circuit4)

1.1916253585419323e-15

In [12]:
get_error(e_ith, U_circuit)

0.49480791850904576

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)